# M4 Dataset Benchmark Code
> Generic code to experiment and produce the final benchmark.py codes

In [5]:
# python
import argparse
import numpy as np
# ml
from sklearn.preprocessing import MinMaxScaler
import mlflow
# local
from models.benchmark import NaivePredictor
from models.cnn import SimpleCNN
from models.transformer import VanillaTransformer, DecoderOnlyTransformer
from utils.plot import plot_predictions
from experiment import Experiment
from utils.m4 import smape, mase, M4DatasetGenerator

## Experiment Definition

In [6]:
TRACK = False

In [7]:
from utils.ml import EarlyStopperPercent
import torch
from utils.m4 import get_x_y
from utils.ml import DecoderDataset

In [8]:
model_name = ['vanilla_transformer','cnn','decoder_transformer'][2]
run_sp = 'Weekly'

assert(model_name in ['cnn','naive', 'vanilla_transformer','decoder_transformer'])
assert run_sp in ['Hourly','Daily','Weekly','Monthly','Quarterly','Yearly']
m4_data = M4DatasetGenerator([run_sp])

def get_model(model_name, model_conf):
    if model_name == 'cnn':
        return SimpleCNN(model_conf['block_size'], model_conf['d_model'])
    elif model_name == 'naive':
        return NaivePredictor()
    elif model_name == 'vanilla_transformer':
        return  VanillaTransformer(model_conf)
    elif model_name == 'decoder_transformer':
        m = DecoderOnlyTransformer(model_conf)
        return torch.load('trained/decoder_only_weekly_38.model').to('cuda')
    
if TRACK:
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    mlflow.set_experiment(f"M4Benchmark {model_name}")
    mlflow.set_experiment_tag('model', model_name)

np.random.seed(123)
#
# Inicializations
#
metrics_table = {'serie_id':[],'smape':[],'mase':[],}
smape_list, mase_list = [], []
num_of_series = m4_data.data_dict[run_sp]['num']
block_size = m4_data.data_dict[run_sp]['fh']
fh = m4_data.data_dict[run_sp]['fh']
#
# Model Hiperparams
#
d_model = 64
batch_size = 512 #512
epochs = 128
scaler = MinMaxScaler((-1,1))
decompose = False
MAX_SERIES = 10
#
# model_conf = {'block_size':block_size, 'd_model':d_model}
if model_name == 'vanilla_transformer':
    model_conf = {'block_size':block_size, 'd_model': 16, 'num_heads': 2, 'num_layers': 2,'dim_feedforward':128,'device':'cuda'}
    lr = 1e-4
elif model_name == 'decoder_transformer':
    model_conf = {
    'd_model': 32, 
    'num_heads': 4, 
    'num_layers': 4,
    'dim_feedforward':128,
    'block_size':512,
    'device':'cuda',
    'pad_token':-20
}
    lr = 1e-4
else:
    lr = 1e-3

#
if TRACK:
    mlflow.start_run(run_name=f'{run_sp}')
    mlflow.log_param('model_name', model_name)
    mlflow.log_param('d_model', d_model)
    mlflow.log_param('block_size', block_size)
    mlflow.log_param('forecast_horizon', fh)
    mlflow.log_param('decompose', decompose)

    mlflow.log_param('series', num_of_series)
    mlflow.log_param('scaler', scaler)
    mlflow.log_param('batch_size', batch_size)
    mlflow.log_param('epochs', epochs)
    mlflow.log_param('lr', lr)
    
for train_serie, test_serie, serie_id, fh, freq, serie_sp in m4_data.generate(n_series=MAX_SERIES, random=True):
    assert fh == block_size
    model = get_model(model_name, model_conf) #
    
    exp_conf = {
            # Model
            'model': model,
            'model_n_parameters': sum(p.numel() for p in model.parameters() if p.requires_grad), 
            'input_len':block_size,
            'forecast_horizon':fh,
            'feature_dim':1,
            # Data
            'frequency':serie_sp.lower(),
            'scaler':scaler,
            'decompose': decompose, #detrend and de-sazonalize
            'freq':freq,
            # Others
            'device':'cuda',
            'verbose':False,
    }
    train_conf = {
        'epochs':epochs,
        'lr':lr, 
        'batch_size':batch_size,
        'validate_freq':10,
        'verbose':False, # stop training if loss dont decrease 0.5% 5 consecutive steps
        'early_stop':EarlyStopperPercent(patience=5, min_percent=0.005, verbose=False),
    }
    if model_name == 'decoder_transformer':
        print(serie_id)
        train_x = torch.tensor(scaler.fit_transform(train_serie.reshape(-1, 1)).reshape(-1), dtype=torch.float32)

        x, y, m = get_x_y(train_x, block_size=512)
        train_conf['train_dataset'] = DecoderDataset(x.unsqueeze(-1), y.unsqueeze(-1), m)
        model.fit(train_conf)

        train_x = train_x.to('cuda').view(1, -1, 1)
        pred_y = model.predict(train_x, len(test_serie)).cpu().numpy()
        pred_y = scaler.inverse_transform(pred_y.reshape(-1,1)).reshape(-1)



    else:
        exp = Experiment(exp_conf)
        exp.set_dataset(linear_serie=train_serie, train=True)
        # exp.set_dataset(linear_serie=test_serie)
        exp.train(train_conf)
        # test
        last_train_values = train_serie[-block_size:]
        pred_y = exp.predict(last_train_values, fh)
    
    # check if negative or extreme (M4)
    pred_y[pred_y < 0] = 0
    pred_y[pred_y > (1000 * np.max(train_serie))] = np.max(train_serie)

    # Metrics
    metrics_table['serie_id'].append(serie_id)
    metrics_table['smape'].append(smape(test_serie, pred_y)*100)
    metrics_table['mase'].append(mase(train_serie, test_serie, pred_y, freq))
    print(f'Serie {serie_id}-{serie_sp} Finished')
    plot_predictions(train_serie, test_serie, pred_y)
    
#
metrics_dict = {
    'smape_mean': np.round(np.mean(metrics_table['smape'], dtype=float), 3), 
    'mase_mean':  np.round(np.mean(metrics_table['mase'], dtype=float), 3),
    #
    'smape_std':  np.round(np.std(metrics_table['smape'], dtype=float), 3),
    'mase_std':   np.round(np.std(metrics_table['mase'], dtype=float), 3),
}
if TRACK:
    mlflow.log_metrics(metrics_dict)
    mlflow.log_table(metrics_table, artifact_file='metrics_table')

print(f'''
    Experiment Finished
''')
for k, v in metrics_dict.items(): print(f'      {k}: {v}')

Loading M4 Data...
Loaded:
    => Weekly has 359 series
W176
